## Weakly Supervised Instance Segmentation using Class Peak Response 
### Evaluation code of object counting in COCO

In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.misc import imresize
import matplotlib.pyplot as plt
from nest import modules, run_tasks
from collections import OrderedDict
import torch.nn.functional as F
from PIL import Image

## make sure you have correctly install cocoapi, https://github.com/cocodataset/cocoapi
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import pycocotools.mask as mask

In [2]:
from sklearn.metrics import mean_squared_error
import torch
def mrmse(non_zero,count_pred, count_gt):
    nzero_mask=torch.ones(count_gt.size())
    if non_zero==1:
        nzero_mask=torch.zeros(count_gt.size())
        nzero_mask[count_gt!=0]=1
    mrmse=torch.pow(count_pred - count_gt, 2)
    mrmse = torch.mul(mrmse, nzero_mask)
    mrmse = torch.sum(mrmse, 0)
    nzero = torch.sum(nzero_mask, 0)
    mrmse = torch.div(mrmse, nzero)
    mrmse = torch.sqrt(mrmse)
#     print(mrmse.size())
    mrmse = torch.mean(mrmse)
    return mrmse

def rel_mrmse(non_zero,count_pred, count_gt):
    nzero_mask=torch.ones(count_gt.size())
    if non_zero==1:
        nzero_mask=torch.zeros(count_gt.size())
        nzero_mask[count_gt!=0]=1
    num = torch.pow(count_pred - count_gt, 2)
    denom = count_gt.clone()
    denom = denom+1
    rel_mrmse = torch.div(num, denom)
    rel_mrmse = torch.mul(rel_mrmse, nzero_mask)
    rel_mrmse = torch.sum(rel_mrmse, 0)
    nzero = torch.sum(nzero_mask, 0)
    rel_mrmse = torch.div(rel_mrmse, nzero)
    rel_mrmse = torch.sqrt(rel_mrmse)
    rel_mrmse = torch.mean(rel_mrmse)
    return rel_mrmse

# image pre-processor
image_size = 448
transformer = modules.image_transform(
    image_size = [image_size, image_size],
    augmentation = dict(),
    mean = [0.485, 0.456, 0.406],
    std = [0.229, 0.224, 0.225])

In [3]:
## load ground truth

coco_path='/raid/xiaoke/MSCOCO/coco'
cocoGt=COCO(coco_path+'/annotations/instances_val2014.json')
image_ids=cocoGt.getImgIds()

catids=cocoGt.getCatIds()

num_classes=len(catids)
catid2index={}
for i,cid in enumerate(catids):
    catid2index[cid]=i
annids=cocoGt.getAnnIds()
class_labels = OrderedDict()
for id in annids:
    anns=cocoGt.loadAnns(id)
    for i in range(len(anns)):
        ann=anns[i]
        name=ann['image_id']
        if name not in class_labels:
            class_labels[name]=np.zeros(num_classes)
        category_id=ann['category_id']
        class_labels[name][catid2index[category_id]]+=1

image_list_all=list(class_labels.keys())
gt_counts_all=[gt[1] for gt in list(class_labels.items())]
gt_counts_all=np.array(gt_counts_all)

loading annotations into memory...
Done (t=4.41s)
creating index...
index created!


In [7]:
## use the first half as validation and second half as test
index=int(40137/2)
image_list=image_list_all[index:]
gt_count=gt_counts_all[index:]

## load model
backbone = modules.fc_resnet50(channels=240, pretrained=False)
model = modules.peak_response_mapping(backbone,enable_peak_stimulation=True,peak_stimulation='addedmodule5',
                                     sub_pixel_locating_factor=1)
model = nn.DataParallel(model)
checkpoint = torch.load('../models/counting/coco14.pt')
model_dict = model.state_dict()
trained_dict = {k: v for k, v in checkpoint['model'].items() if k in model_dict}
model_dict.update(trained_dict)
model.load_state_dict(model_dict)
model = model.module.cuda()
#print(model)


<function PeakResponseMapping._median_filter at 0x7ff46b559048>
addedmodule5
enable_peak_stimulation on


In [8]:
## counting
pred_class=[]
pred_count=[]
for index_d,ima in enumerate(image_list):
    if index_d%10000==0:
        print(index_d)
    raw_img = Image.open(coco_path+'/images/val2014/COCO_val2014_'+'0'*(12-len(str(ima)))+str(ima)+'.jpg').convert('RGB')
    width, height=raw_img.size
    input_var = transformer(raw_img).unsqueeze(0).cuda().requires_grad_()
    model = model.eval()
    confidence,class_response_map1,peak = model(input_var,1)
    confidence=confidence.cpu().detach().numpy()
    count_one = F.adaptive_avg_pool2d(class_response_map1, 1).squeeze(2).squeeze(2).detach().cpu().numpy()[0]
    confidence[confidence<0]=0
    confidence=confidence[0]
    confidence[confidence>0]=1
    pred_class.append(confidence)
    pred_count.append(np.round(confidence*count_one))
pred_count=np.array(pred_count)
print(mrmse(1,torch.from_numpy(pred_count).float(), torch.from_numpy(gt_count).float()))
print(rel_mrmse(1,torch.from_numpy(pred_count).float(), torch.from_numpy(gt_count).float()))
print(mrmse(0,torch.from_numpy(pred_count).float(), torch.from_numpy(gt_count).float()))
print(rel_mrmse(0,torch.from_numpy(pred_count).float(), torch.from_numpy(gt_count).float()))

0
10000
20000
tensor(1.9086)
tensor(0.8489)
tensor(0.3397)
tensor(0.1788)
